In [1]:
import os

from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate
)


from langchain_core.messages import AIMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function

from langchain.memory import ConversationBufferMemory
from langchain.agents import tool, AgentExecutor
import json



import requests

In [2]:
OPENAI_API_KEY = "95058a9e99794e4689d179dd726e7eec"
OPENAI_DEPLOYMENT_NAME = "vassar-turbo35"
OPENAI_API_BASE = "https://vassar-openai.openai.azure.com/"
MODEL_NAME = "gpt-35-turbo" 
OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = "2023-07-01-preview"

gpt3_llm = AzureChatOpenAI(
    deployment_name=OPENAI_DEPLOYMENT_NAME,
    model_name=MODEL_NAME,
    openai_api_base=OPENAI_API_BASE,
    openai_api_type=OPENAI_API_TYPE,
    openai_api_key=OPENAI_API_KEY,
    openai_api_version=OPENAI_API_VERSION,
    temperature=0.0,
)

/home/premkumar/ccp-venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/home/premkumar/ccp-venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://vassar-openai.openai.azure.com/ to https://vassar-openai.openai.azure.com/openai.
  warnings.warn(
/home/premkumar/ccp-venv/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:174: Us

In [46]:
AMERICAN_YACHT_GROUP_PROMPT = """
You are Vani, the Customer Support Sales Assistant for American Yacht Group, dedicated to helping customers find their dream yacht. Your mission is to efficiently gather essential details for tailored yacht recommendations, responding within 5-10 words.

Follow these steps:

1. Warmly greet the customer and extend a hearty welcome to American Yacht Group.

2. Thoroughly understand and acknowledge the customer's input regarding Budget Constraints, Desired Length Range, and Location Preference simultaneously.

3. Utilize the "get_filtered_yachts" tool to retrieve relevant yachts after gathering this information. Convert length units to feet before inputting into the tool.  Respond back to the customer with the number of results found and updated on screen.

4. To further refine the filtered results, inquire about Additional Preferences by prompting them: Boat type, preferred builders, boat condition, number of engines, and fuel type to enhance the search.

5. Before calling the 'get_filtered_yachts' tool, provide all the gathered inputs, including Budget Constraints, Desired Length Range, Location Preference, and any Additional Preferences like Boat type, preferred builders, boat condition, number of engines, and fuel type. Convert length units to feet before inputting into the tool. Respond back to the customer with the number of results found and updated on screen.

Maintain a friendly and engaging tone throughout the interaction.

For a natural interaction, structure your conversation accordingly:
- Respond to user queries based on the steps above.
- Gradually ask for additional details after completing each previous step.
- Provide information and assistance based on user requests.
- Output in a visually appealing format.

Remember to adapt your responses based on the user's input to create a seamless and engaging conversation.
"""


In [47]:
american_yacht_group_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(AMERICAN_YACHT_GROUP_PROMPT),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


In [48]:

AYG_URL = "https://services.boats.com/pls/boats/search"
FIELDS = "BuilderName,DocumentID,EmbeddedVideo,EmbeddedVideoPresent,NominalLength,LengthOverall,IsAvailableForPls,StockNumber,MaximumSpeedMeasure,FuelTankCapacityMeasure,DryWeightMeasure,engines,BeamMeasure,NumberOfEngines,HoldingTankCapacityMeasure,WaterTankCapacityMeasure,ModelYear,MakeString,Model,Images,BoatName,BoatLocation,SaleClassCode,BoatClassCode,Office,PriceHideInd,Price,GeneralBoatDescription,AdditionalDetailDescription"
KEY = "gs4g3hpp688c"

In [49]:
def get_yachts_api(filter_params=None):
    api_url = AYG_URL
    params = {'fields': FIELDS, 'key': KEY, 'rows': '10', 'offset': 0   }

    # If filter_params is provided, add them to the params dictionary
    if filter_params:
        params.update({key: value for key, value in filter_params.items() if value is not None})

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()

        # If the response is in JSON format, you can access the data like this:
        inventory_details = response.json()

        # Convert the Python dictionary to a JSON-formatted string
        inventory_json_details = json.dumps(inventory_details, indent=2)

        # print("inventory details :: ", inventory_json_details)

        # Parse the JSON string back into a Python dictionary before using it with 'get'
        parsed_data = json.loads(inventory_json_details)

        return parsed_data
    except requests.exceptions.RequestException as e:
        # Handle any exceptions that might occur during the request
        print(f"Error making API request: {e}")
        return None
    
def process_api_data(api_data):
    numResults = api_data.get("data", {}).get("numResults", 0)
    boats_data = api_data.get("data", {}).get("results", [])

    print(f"No. of results from API: {len(boats_data)}")

    boats_json = []

    for boat_data in boats_data:

        document_id = boat_data.get("DocumentID", "")
        builder_name = boat_data.get('BuilderName', '')
        boat_name = boat_data.get('BoatName', '')
        location_data = boat_data.get('BoatLocation', {})
        price = boat_data.get('Price', None)
        engines_data = boat_data.get('Engines', [])
        number_of_engines = len(engines_data)
        images = boat_data.get('Images', '')
        nominal_length = boat_data.get('NominalLength', '')
        length_overall = boat_data.get('LengthOverall', '')
        model_name = boat_data.get('Model', '')
        model_year = boat_data.get('ModelYear', '')
        boat_type = boat_data.get('category', '')
        condition = boat_data.get('SaleClassCode', '')

        # Set a default price if it is None
        if price is None:
            price = 'CALL FOR PRICE'

        # Insert data into Location table
        location = {
            'country': location_data.get('BoatCountryID', ''),
            'state': location_data.get('BoatStateCode', ''),
            'city': location_data.get('BoatCityName', '')
        }

        # Insert or get data from Boat table
        boat_instance = {
            'document_id': document_id,
            'builder_name': builder_name,
            'boat_name': boat_name,
            'boat_type': boat_type,
            'location': location,
            'price': price,
            'images': images[:5],
            'number_of_engines': number_of_engines,
            'nominal_length': nominal_length,
            'length_overall': length_overall,
            'model_name': model_name,
            'model_year': model_year,
            'condition': condition,
            'engines': []
        }

        # Insert data into Engine table for each engine in the list
        for engine_data in engines_data:
            engine = {
                'engine_make': engine_data.get('Make', ''),
                'engine_model': engine_data.get('Model', ''),
                'drive_transmission_description': engine_data.get('DriveTransmissionDescription', ''),
                'fuel': engine_data.get('Fuel', ''),
                'engine_power': engine_data.get('EnginePower', ''),
                'engine_type': engine_data.get('Type', ''),
                'propeller_type': engine_data.get('PropellerType', ''),
                'engine_year': engine_data.get('Year', 0),
                'engine_hours': engine_data.get('Hours', 0),
                'boat_engine_location_code': engine_data.get('BoatEngineLocationCode', '')
            }
            boat_instance['engines'].append(engine)

        boats_json.append(boat_instance)

    boat_details = {
        "results": numResults,
        "boat_data": boats_json
    }
    return boat_details



In [61]:
@tool
def get_filtered_yachts(builder=None, boat_type=None, boat_condition=None, city=None, state=None, country=None, model_year=None, price_min=0, price_max=999999999, boat_name=None, min_length=25, max_length=400, num_engines=None, fuel_type:str=''):
    """
    Use this tool to display yachts based on specified filters.
    boat_condition: str='New|Used', country: str = 'US|UK|IND', fuel_type: str ='diesel|electric'

    Returns:
    - A message indicating the number of results found with the given filters and an update about the filtered yachts on the screen interface.
    - An error message if there is no response from the API.
    """

    if price_max==price_min:
        price_min = 0.9*price_min
        price_max = 1.1*price_max

    
    # Define a dictionary with parameters and their values
    filter_params = {'BuilderName': builder, 'city': city, 'state': state, 'country': country, 'ModelYear': model_year,
                     'BoatName': boat_name, 'length': f'{min_length}:{max_length}|feet', 'price': f'{price_min}:{price_max}', "class": boat_type, "condition": boat_condition, 'engines': num_engines, 'fuel': fuel_type.lower()}

    # Filter out parameters with None values and Making BuilderName from list to string.
    filter_params = {
                    key: (','.join(value) if key == 'BuilderName' and isinstance(value, list) else value)
                    for key, value in filter_params.items() if value is not None and value!=''}


    # Print filtered parameters for debugging
    print("filter params",filter_params)

    # Call API to retrieve yachts data
    api_data = get_yachts_api(filter_params=filter_params)

    # Process API data
    if api_data:
        boat_details = process_api_data(api_data=api_data)

        # print(f"Boat Details::: {boat_details.get('results')}")

        # Update session's boat_list field

        return f"Found {boat_details.get('results', 0)} yachts matching the specified filters: [{filter_params}]. Inform customer that filtered results have been updated on the screen."
    
    # Return error message if no response from API
    return "Error from API Response"

In [62]:
tools = [get_filtered_yachts]
# tools = [get_yachts]

In [63]:
llm_with_tools = gpt3_llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])


agent = (
        {
            "question": lambda x: x["question"],
            "chat_history": lambda x: x["chat_history"],
            "agent_scratchpad": lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            )
        }
        | american_yacht_group_prompt
        | llm_with_tools
        | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [64]:
chat_history = []

In [67]:
memory = ConversationBufferMemory(return_messages=True)

question = input("Query: ")

result = agent_executor({
            "question": question,
            "chat_history": chat_history
        })
chat_history.extend([HumanMessage(content=question),AIMessage(content=result['output'])])
print("result:", result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_filtered_yachts` with `{'price_min': 1000000, 'price_max': 1000000, 'city': 'Miami'}`


filter params {'city': 'Miami', 'length': '25:400|feet', 'price': '900000.0:1100000.0'}
No. of results from API: 10
Found 35 yachts matching the specified filters: [{'city': 'Miami', 'length': '25:400|feet', 'price': '900000.0:1100000.0'}]. Inform customer that filtered results have been updated on the screen.Great! I found 35 yachts in Miami within your budget of $1 million. The filtered results have been updated on the screen. 

Now, to further refine the search, could you please provide me with any additional preferences you have? This could include boat type, preferred builders, boat condition, number of engines, and fuel type.

> Finished chain.
result: Great! I found 35 yachts in Miami within your budget of $1 million. The filtered results have been updated on the screen. 

Now, to further refine the search, could you please provide me wi